In [ ]:
#https://towardsdatascience.com/build-a-super-simple-gan-in-pytorch-54ba349920e4


In [242]:
import torch
import torchvision
from torchvision import transforms, datasets, models

import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F



import os 
import time
import cv2

from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

rebuild_data = False

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda: 0")
else:
    device = torch.device("cpu")
    
#i could assign specific layers to one gpu 

torch.cuda.device_count() #but i have only one :P
    

1

In [244]:
def create_binary_list_from_int(number):
    return [int(x) for x in list(bin(number))[2:]]

In [9]:
bin(4)

'0b100'

In [10]:
dec2bin(4)

[1, 0, 0]

In [220]:
def makeEvenData(max_int, batch_size = 16):
    
    max_length = int(math.log(max_int, 2))
    
    sampled_integers = np.random.randint(0, int(max_int /2), batch_size)
    
    labels = [1] * batch_size
    
    data = [create_binary_list_from_int(int(x*2)) for x in sampled_integers]
    data = [([0] * (max_length - len(x))) + x for x in data]
    
    return labels, data

In [221]:
[0]+[1,2]

[0, 1, 2]

In [222]:
np.random.rand(2)

array([0.03813136, 0.96011237])

In [223]:
np.random.randint(0, 4, 16)

array([0, 3, 3, 0, 2, 3, 2, 2, 2, 1, 3, 3, 3, 3, 1, 0])

In [230]:
class Generator(nn.Module):
    
    def __init__(self, input_length):
        super().__init__()
        self.fc = nn.Linear(int(input_length), int(input_length))
    
    def forward(self, x):
        return F.sigmoid(self.fc(x))
 

In [231]:
'''If we were building a GAN to do something more 
complicated on say images we would probably train 
it using random noise generated from a normal 
distribution and gradually upsample and reshape 
it until it’s the same size as the data we are trying to copy. '''

'If we were building a GAN to do something more \ncomplicated on say images we would probably train \nit using random noise generated from a normal \ndistribution and gradually upsample and reshape \nit until it’s the same size as the data we are trying to copy. '

In [232]:
# 2 dots, 4 noodles so to speak
net = Generator(2)

list(net.parameters())

[Parameter containing:
 tensor([[ 0.4901,  0.6977],
         [ 0.0996, -0.3970]], requires_grad=True), Parameter containing:
 tensor([ 0.3716, -0.2405], requires_grad=True)]

In [233]:
class Discriminator(nn.Module):
    
    def __init__(self, input_length):
        super().__init__()
        self.fc = nn.Linear(int(input_length), 1)
        
    def forward(self, x):
        return F.sigmoid(self.fc(x))
    
    
    

In [234]:
net = Discriminator(2)
list(net.parameters())
#expect 1 dot, 3 noodles

[Parameter containing:
 tensor([[-0.3078, -0.3047]], requires_grad=True), Parameter containing:
 tensor([-0.1248], requires_grad=True)]

In [ ]:
'''we pass two batches of data to our model 
at every training step. One batch is random 
noise which will cause the generator to create 
some generated data, and the second batch is
composed solely of 
data from our true distribution. '''

In [238]:
torch.randint(0, 2, size=(2, 5)).float().shape

torch.Size([2, 5])

In [250]:
def train(max_int = 128, batch_size = 16, training_steps = 500):
    
    input_length = int(math.log(max_int, 2))
    
    generator = Generator(input_length).to(device)
    discriminator = Discriminator(input_length).to(device)
    
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr = 0.001)
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr = 0.001)
    
    loss_function = nn.BCELoss()
    
    for i in range(training_steps):
        
        
       
        
        noise = torch.randint(0, 2, size =(batch_size, input_length)).float()
        
        #####################################################
        generator_optimizer.zero_grad()
        
        generated_data = generator(noise.to(device)) #
        
        true_labels, true_data = makeEvenData(max_int, batch_size = batch_size)
        true_labels = torch.tensor(true_labels).float().to(device)
        true_data = torch.tensor(true_data).float().to(device)
        
        
        
        generator_discriminator_out = discriminator(generated_data) #
        generator_loss = loss_function(generator_discriminator_out, true_labels)#note true_labels is nothing but torch.ones(batch_size)
        generator_loss.backward()
        generator_optimizer.step()
        
        ######################################################         
        discriminator_optimizer.zero_grad()
        
        true_discriminator_out = discriminator(true_data)
        true_discriminator_loss = loss_function(true_discriminator_out, true_labels)
        
        ######################################################
        #why did i detach it? and why can't i type this night AA A FUUUUUU
        
        
        #http://www.bnikolic.co.uk/blog/pytorch-detach.html
        
        
        generator_discriminator_out = discriminator(generated_data.detach().to(device))
        generator_discriminator_loss = loss_function(generator_discriminator_out, torch.zeros(batch_size).to(device))
        discriminator_loss = (true_discriminator_loss + generator_discriminator_loss) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()
        print(generator(torch.randint(0, 2, size =(16, 7)).float().to(device)))
    
        
        
        

In [251]:
#NOT WORKING RETRY LATER


tensor([[0.3458, 0.7332, 0.4655, 0.4336, 0.5688, 0.6612, 0.5024],
        [0.3750, 0.5240, 0.5315, 0.4146, 0.4754, 0.4363, 0.5476],
        [0.4376, 0.5909, 0.4598, 0.3998, 0.5524, 0.4404, 0.4000],
        [0.3477, 0.5607, 0.5641, 0.3758, 0.5599, 0.4423, 0.5772],
        [0.3574, 0.6372, 0.5663, 0.3893, 0.5884, 0.4381, 0.5108],
        [0.3700, 0.7397, 0.4992, 0.4348, 0.5678, 0.6035, 0.4255],
        [0.4190, 0.6412, 0.5343, 0.3552, 0.5857, 0.4282, 0.4343],
        [0.4561, 0.5083, 0.6721, 0.5762, 0.4709, 0.3892, 0.4321],
        [0.2583, 0.7298, 0.5162, 0.3773, 0.6262, 0.6232, 0.5284],
        [0.2705, 0.6699, 0.5476, 0.3650, 0.5976, 0.5676, 0.5180],
        [0.5440, 0.4943, 0.5523, 0.5226, 0.4626, 0.3742, 0.4324],
        [0.3428, 0.7671, 0.5320, 0.3953, 0.6485, 0.6094, 0.4552],
        [0.4318, 0.6335, 0.4821, 0.4450, 0.5305, 0.5411, 0.5618],
        [0.3696, 0.6649, 0.4547, 0.3174, 0.5887, 0.4901, 0.4412],
        [0.3848, 0.6810, 0.5306, 0.4219, 0.5381, 0.5471, 0.4153],
        [0

       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.3340, 0.7741, 0.5222, 0.3859, 0.6394, 0.6000, 0.4650],
        [0.3511, 0.7348, 0.4478, 0.4318, 0.6076, 0.6119, 0.4301],
        [0.3719, 0.6928, 0.5170, 0.4086, 0.5245, 0.5336, 0.4287],
        [0.4753, 0.4939, 0.5803, 0.5551, 0.4616, 0.3800, 0.5127],
        [0.4484, 0.5161, 0.6652, 0.5686, 0.4632, 0.3818, 0.4397],
        [0.4656, 0.6411, 0.5002, 0.4464, 0.5713, 0.4283, 0.4108],
        [0.4076, 0.6519, 0.5226, 0.3446, 0.5742, 0.4168, 0.4459],
        [0.3697, 0.5787, 0.6222, 0.4859, 0.5508, 0.4481, 0.4767],
        [0.3203, 0.7516, 0.5020, 0.3307, 0.6078, 0.5380, 0.3916],
        [0.3935, 0.4954, 0.5711, 0.4492, 0.4674, 0.4302, 0.6216],
        [0.4981, 0.5611, 0.4833, 0.5306, 0.4896, 0.4874, 0.5238],
        [0.3444, 0.7346, 0.4123, 0.3187, 0.6063, 0.5360, 0.4632],
        [0.3260, 0.6645, 0.6452, 0.3950, 0.5824, 0.4323, 0.4456],
        [0.3283, 0.6350, 0.5270, 0.3826, 0.5835, 0.4939, 0.5931],
        [0.4109, 0.5748, 

       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.4071, 0.5783, 0.6046, 0.5801, 0.4939, 0.4993, 0.5275],
        [0.4355, 0.6367, 0.4623, 0.4412, 0.5682, 0.4858, 0.4914],
        [0.3243, 0.6773, 0.3986, 0.2964, 0.5664, 0.5285, 0.5419],
        [0.3461, 0.6086, 0.4844, 0.4123, 0.4897, 0.4781, 0.5732],
        [0.4387, 0.5939, 0.3971, 0.4804, 0.4847, 0.5416, 0.5387],
        [0.3546, 0.6073, 0.5225, 0.5345, 0.4931, 0.5574, 0.5383],
        [0.3338, 0.7428, 0.4008, 0.3085, 0.5948, 0.5242, 0.4749],
        [0.3421, 0.7417, 0.4380, 0.4221, 0.5981, 0.6024, 0.4398],
        [0.2869, 0.7208, 0.5795, 0.4026, 0.6083, 0.5476, 0.5372],
        [0.3939, 0.7181, 0.5213, 0.4678, 0.5989, 0.5453, 0.4291],
        [0.3780, 0.6085, 0.4150, 0.3725, 0.4838, 0.5260, 0.5778],
        [0.4084, 0.6185, 0.4822, 0.4902, 0.4823, 0.5396, 0.4695],
        [0.2896, 0.6830, 0.5135, 0.4411, 0.5261, 0.6022, 0.5838],
        [0.4539, 0.5912, 0.5277, 0.4140, 0.4798, 0.4029, 0.4939],
        [0.4539, 0.5912, 

       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.3773, 0.5593, 0.5697, 0.5051, 0.4445, 0.4208, 0.4682],
        [0.4568, 0.4209, 0.5703, 0.5330, 0.4250, 0.3766, 0.5868],
        [0.4599, 0.5897, 0.5101, 0.4126, 0.5201, 0.3530, 0.4216],
        [0.3044, 0.6203, 0.5234, 0.4228, 0.5323, 0.4887, 0.5052],
        [0.3058, 0.6804, 0.6234, 0.3730, 0.5593, 0.4095, 0.4688],
        [0.2975, 0.7353, 0.5991, 0.3909, 0.5943, 0.4721, 0.4733],
        [0.3940, 0.6800, 0.4002, 0.3846, 0.5530, 0.4686, 0.4387],
        [0.4005, 0.6843, 0.5745, 0.4289, 0.4996, 0.4498, 0.4431],
        [0.4675, 0.5000, 0.5726, 0.5473, 0.4537, 0.3726, 0.5206],
        [0.4533, 0.6103, 0.4184, 0.4700, 0.4720, 0.4680, 0.4730],
        [0.4005, 0.6843, 0.5745, 0.4289, 0.4996, 0.4498, 0.4431],
        [0.4157, 0.6775, 0.5668, 0.4368, 0.5495, 0.4073, 0.3637],
        [0.2938, 0.7692, 0.4706, 0.3038, 0.5774, 0.5067, 0.4218],
        [0.5001, 0.5011, 0.5010, 0.5038, 0.4459, 0.4166, 0.5272],
        [0.3732, 0.5704, 

       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.3538, 0.5413, 0.4848, 0.3855, 0.4864, 0.3578, 0.5038],
        [0.2858, 0.6231, 0.4720, 0.2982, 0.5157, 0.3975, 0.5535],
        [0.2491, 0.7073, 0.5269, 0.3177, 0.5432, 0.4518, 0.4949],
        [0.4716, 0.4341, 0.5919, 0.5228, 0.4127, 0.3104, 0.5218],
        [0.3347, 0.7110, 0.4759, 0.3701, 0.4832, 0.4928, 0.4692],
        [0.3484, 0.5907, 0.6000, 0.4630, 0.5275, 0.4253, 0.4999],
        [0.3603, 0.6147, 0.3964, 0.3550, 0.4645, 0.5070, 0.5964],
        [0.2810, 0.6691, 0.5876, 0.3687, 0.5566, 0.4671, 0.5496],
        [0.3324, 0.6136, 0.4688, 0.3976, 0.4741, 0.4627, 0.5883],
        [0.3772, 0.5046, 0.5540, 0.4324, 0.4501, 0.4134, 0.6377],
        [0.3324, 0.6136, 0.4688, 0.3976, 0.4741, 0.4627, 0.5883],
        [0.3275, 0.7623, 0.4522, 0.3896, 0.5210, 0.5579, 0.4719],
        [0.3369, 0.5933, 0.5593, 0.3417, 0.5205, 0.3466, 0.5388],
        [0.3370, 0.5281, 0.4588, 0.3917, 0.4952, 0.4242, 0.5730],
        [0.3437, 0.6719, 

...

       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.3543, 0.4653, 0.3790, 0.6429, 0.2925, 0.4217, 0.5991],
        [0.2053, 0.6086, 0.3445, 0.6527, 0.3187, 0.5151, 0.5761],
        [0.1682, 0.7009, 0.2658, 0.5858, 0.3442, 0.7115, 0.7004],
        [0.0980, 0.6905, 0.2096, 0.5866, 0.2762, 0.6462, 0.6969],
        [0.2809, 0.6127, 0.3458, 0.5699, 0.3509, 0.5280, 0.6477],
        [0.2193, 0.6061, 0.3529, 0.6222, 0.3595, 0.5630, 0.6111],
        [0.2673, 0.4617, 0.4289, 0.5888, 0.3451, 0.3828, 0.6224],
        [0.1667, 0.7367, 0.3208, 0.5480, 0.4232, 0.6640, 0.6591],
        [0.1875, 0.5553, 0.2354, 0.5761, 0.2523, 0.5564, 0.7165],
        [0.1554, 0.7387, 0.3129, 0.5804, 0.3795, 0.6197, 0.6257],
        [0.1058, 0.6283, 0.2979, 0.5470, 0.3312, 0.6361, 0.7448],
        [0.1667, 0.7367, 0.3208, 0.5480, 0.4232, 0.6640, 0.6591],
        [0.2327, 0.5558, 0.3335, 0.5277, 0.4184, 0.5484, 0.6716],
        [0.2193, 0.6061, 0.3529, 0.6222, 0.3595, 0.5630, 0.6111],
        [0.2222, 0.5638, 

       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.1906, 0.5655, 0.2429, 0.5867, 0.2593, 0.5631, 0.7097],
        [0.3093, 0.6288, 0.4153, 0.6873, 0.3546, 0.5462, 0.5390],
        [0.2013, 0.5373, 0.3981, 0.5053, 0.3856, 0.4948, 0.7218],
        [0.1358, 0.6280, 0.2050, 0.5396, 0.2934, 0.5826, 0.7095],
        [0.3574, 0.4722, 0.3854, 0.6494, 0.2976, 0.4262, 0.5937],
        [0.1354, 0.5620, 0.2832, 0.5304, 0.3086, 0.5229, 0.7294],
        [0.2584, 0.4766, 0.4439, 0.5870, 0.3073, 0.4282, 0.6909],
        [0.3148, 0.4963, 0.3886, 0.5806, 0.3822, 0.5320, 0.6680],
        [0.2081, 0.5517, 0.4562, 0.6541, 0.3428, 0.4615, 0.5921],
        [0.2128, 0.4851, 0.3289, 0.5527, 0.3534, 0.5076, 0.6935],
        [0.1702, 0.6482, 0.3646, 0.5857, 0.3679, 0.6645, 0.7151],
        [0.2203, 0.5652, 0.3314, 0.5676, 0.3808, 0.5049, 0.6336],
        [0.2030, 0.4931, 0.3362, 0.5437, 0.3098, 0.5500, 0.7583],
        [0.2245, 0.6346, 0.2564, 0.5334, 0.3593, 0.6502, 0.7163],
        [0.2133, 0.5535, 

       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.1914, 0.5486, 0.3995, 0.5481, 0.3521, 0.4526, 0.6843],
        [0.1636, 0.5750, 0.3557, 0.6209, 0.3529, 0.5877, 0.6643],
        [0.1149, 0.7689, 0.2351, 0.5929, 0.3605, 0.6991, 0.6520],
        [0.1542, 0.6252, 0.4270, 0.5753, 0.3851, 0.5753, 0.6941],
        [0.3179, 0.5689, 0.2979, 0.5944, 0.3700, 0.5943, 0.6415],
        [0.3125, 0.6340, 0.4224, 0.6928, 0.3605, 0.5500, 0.5330],
        [0.2856, 0.4925, 0.5178, 0.7014, 0.3109, 0.4455, 0.5867],
        [0.2025, 0.6338, 0.3716, 0.6630, 0.2931, 0.5697, 0.6376],
        [0.1931, 0.5045, 0.3376, 0.5858, 0.2799, 0.5078, 0.7239],
        [0.1723, 0.6533, 0.3713, 0.5921, 0.3740, 0.6678, 0.7103],
        [0.2055, 0.4989, 0.3428, 0.5504, 0.3155, 0.5538, 0.7540],
        [0.2461, 0.6255, 0.4795, 0.6082, 0.4600, 0.5556, 0.5955],
        [0.2216, 0.6381, 0.4820, 0.6338, 0.3711, 0.5520, 0.6359],
        [0.1610, 0.7528, 0.3314, 0.6004, 0.3976, 0.6313, 0.6095],
        [0.2274, 0.5786, 

       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.2283, 0.6838, 0.3237, 0.5083, 0.4532, 0.6052, 0.6650],
        [0.1062, 0.7192, 0.2402, 0.6248, 0.3088, 0.6670, 0.6658],
        [0.4100, 0.5494, 0.4102, 0.6050, 0.4111, 0.5208, 0.5952],
        [0.1629, 0.7018, 0.4393, 0.5972, 0.4217, 0.5764, 0.6269],
        [0.2281, 0.5765, 0.3558, 0.5339, 0.3855, 0.5984, 0.7289],
        [0.2166, 0.6329, 0.3751, 0.6792, 0.3450, 0.5325, 0.5494],
        [0.1965, 0.6234, 0.3147, 0.5682, 0.3470, 0.5188, 0.6523],
        [0.3649, 0.4828, 0.4001, 0.6610, 0.3092, 0.4332, 0.5815],
        [0.1123, 0.6521, 0.3262, 0.5767, 0.3579, 0.6519, 0.7239],
        [0.2154, 0.5831, 0.3522, 0.5710, 0.3475, 0.5541, 0.6956],
        [0.2914, 0.5681, 0.4237, 0.7168, 0.3045, 0.5108, 0.5529],
        [0.2957, 0.4994, 0.2973, 0.6207, 0.3118, 0.5550, 0.6612],
        [0.1179, 0.7746, 0.2450, 0.6032, 0.3717, 0.7037, 0.6419],
        [0.2258, 0.5753, 0.3448, 0.5800, 0.3935, 0.5118, 0.6221],
        [0.1644, 0.7579, 

       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.1719, 0.6533, 0.2861, 0.6479, 0.3580, 0.6536, 0.6215],
        [0.1588, 0.6512, 0.3165, 0.5342, 0.3994, 0.5785, 0.6852],
        [0.4084, 0.4792, 0.5140, 0.5943, 0.4269, 0.4593, 0.6203],
        [0.1673, 0.6674, 0.3847, 0.6400, 0.3509, 0.6321, 0.6627],
        [0.1531, 0.5754, 0.3143, 0.5196, 0.3717, 0.5814, 0.7400],
        [0.3957, 0.5588, 0.4125, 0.6442, 0.3773, 0.4773, 0.5504],
        [0.3066, 0.6250, 0.3664, 0.6039, 0.4235, 0.4997, 0.5451],
        [0.3236, 0.5751, 0.3080, 0.6031, 0.3801, 0.5984, 0.6323],
        [0.2195, 0.6935, 0.3297, 0.5530, 0.4222, 0.5642, 0.6197],
        [0.1771, 0.6607, 0.3864, 0.6034, 0.3873, 0.6727, 0.6995],
        [0.1622, 0.6991, 0.3512, 0.6036, 0.3906, 0.6420, 0.6531],
        [0.3236, 0.5751, 0.3080, 0.6031, 0.3801, 0.5984, 0.6323],
        [0.2184, 0.5866, 0.3599, 0.5766, 0.3542, 0.5565, 0.6897],
        [0.1885, 0.7158, 0.2896, 0.6254, 0.4229, 0.6905, 0.5980],
        [0.2024, 0.5864, 